In [1]:
import requests
from bs4 import BeautifulSoup
import pickle

ALL_CLASSES = 'http://nwmissouri.smartcatalogiq.com/2020-2021/Undergraduate-Catalog/Courses/CSIS-Computer-Science-Info-Systems'
PROGRAM_REQS = 'http://nwmissouri.smartcatalogiq.com/2020-2021/Undergraduate-Catalog/School-of-Computer-Science-and-Information-Systems/Computer-Science-and-Information-Systems-44/Computer-Science-Comprehensive-Major-6669-hours-BSNo-Minor-Required'
BASE_URL='http://nwmissouri.smartcatalogiq.com'

In [67]:
# get list of program requirements
reqpage = requests.get(PROGRAM_REQS)
reqsoup = BeautifulSoup(reqpage.text, 'html5lib')
reqlist = [x.text.split()[1] for x in reqsoup.findAll('a', {'class': 'sc-courselink'}) if x.text.startswith('CSIS')]
with open('csugrad.pkl', 'wb') as f:
    pickle.dump(reqlist, f)

In [2]:
# build prereq dependencies
classpage = requests.get(ALL_CLASSES)
classsoup = BeautifulSoup(classpage.text, 'html5lib')
classlist = classsoup.find('ul', {'class': 'sc-child-item-links'})

In [20]:
cnums = [x.text.split()[1] for x in classlist.findAll('a') if x.text.startswith('CSIS')]
cnames = {x.text.split()[1]: ' '.join(x.text.split()[2:]) for x in classlist.findAll('a') if x.text.startswith('CSIS')}
#curls = {x.text.split()[1]: BASE_URL+x['href'] for x in classlist.findAll('a') if x.text.startswith('CSIS')}
curls = {x.text.split()[1]: BASE_URL+x['href'] for x in classlist.findAll('a')}

In [46]:
dspage = requests.get(curls['44242'])

In [47]:
dssoup = BeautifulSoup(dspage.text)

In [7]:
prereqs = [x.text.split()[1] for x in dssoup.findAll('a', {'class':'sc-courselink'}) if x.text.startswith('CSIS')]

NameError: name 'dssoup' is not defined

In [52]:
prereqs

['44241']

In [6]:
def pull_prereqs(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html5lib')
    #return [x.text.split()[1] for x in soup.findAll('a', {'class':'sc-courselink'}) if x.text.startswith('CSIS')]
    return [x.text.split()[1] for x in soup.findAll('a', {'class':'sc-courselink'})]

In [21]:
prereqs = {c: pull_prereqs(curls[c]) for c in curls}

In [10]:
with open ('prereqs.pkl', 'wb') as f:
    pickle.dump(prereqs, f)

In [22]:
prereqs

{'44100': [],
 '44101': [],
 '44125': [],
 '44130': [],
 '44141': ['44101', '44130'],
 '44144': ['44101', '44130', '62130'],
 '44149': ['17117', '17120'],
 '44155': [],
 '44181': ['44101', '44130'],
 '44200': [],
 '44241': ['44141'],
 '44242': ['44241', '17114', '17117', '17118', '17120'],
 '44249': ['44141', '17117', '17118', '17120'],
 '44296': ['44141'],
 '44300': [],
 '44312': ['44101', '44130', '62130'],
 '44317': ['54313', '44101', '44130'],
 '44320': ['44101', '44130', '62130'],
 '44330': ['44101', '44130', '62130'],
 '44334': ['44144'],
 '44335': ['44334', '44141', '44144'],
 '44336': ['44101', '44130', '62130'],
 '44337': ['44336'],
 '44340': ['44101', '44130', '62130'],
 '44343': ['44101', '44130', '62130'],
 '44345': ['44242'],
 '44346': ['44101', '44130', '62130'],
 '44349': ['44242', '44249'],
 '44351': ['44242'],
 '44356': ['44130', '44101', '44542', '44542'],
 '44359': ['44356'],
 '44382': ['44242', '44356'],
 '44386': ['44241', '44296', '44356'],
 '44400': [],
 '44410':

In [58]:
import json
with open('prereqs.json', 'w') as f:
    json.dump(prereqs, f)
with open('csugrad.json', 'w') as f:
    json.dump(reqlist, f)


In [59]:
bscs_prereqs = {c: prereqs[c] for c in reqlist}

In [61]:
with open('csonly_prereqs.json', 'w') as f:
    json.dump(bscs_prereqs, f)

In [64]:
import json
with open('csonly_prereqs.json', 'r') as f:
    bscs_prereqs = json.load(f)
    

In [71]:
import pickle
with open('cnames.pkl', 'wb') as f:
    pickle.dump(cnames, f)

In [11]:
nodes = [{'id': c, 'name': cnames.get(c, '') , 'group': int(c[2])} for c in prereqs]

In [12]:
links = []
for c in prereqs:
    for pr in prereqs[c]:
        if pr[2] != '6':
            links.append({'source': pr, 'target': c, 'value': 1})

In [14]:
import json
with open('csprereqs_arc.json', 'w') as f:
    json.dump({'nodes': nodes, 'links': links}, f)

In [16]:
nodes

[{'id': '44100', 'name': 'Special Offering', 'group': 1},
 {'id': '44101', 'name': 'Foundations of Computing', 'group': 1},
 {'id': '44125', 'name': 'Special Topics', 'group': 1},
 {'id': '44130', 'name': 'Computers and Information Technology', 'group': 1},
 {'id': '44141', 'name': 'Computer Programming I', 'group': 1},
 {'id': '44144', 'name': 'Web Development I', 'group': 1},
 {'id': '44149', 'name': 'Scientific Computing', 'group': 1},
 {'id': '44155',
  'name': 'Introduction to Data Science and Data Mining',
  'group': 1},
 {'id': '44181', 'name': 'Introduction to Cybersecurity', 'group': 1},
 {'id': '44200', 'name': 'Special Offering', 'group': 2},
 {'id': '44241', 'name': 'Computer Programming II', 'group': 2},
 {'id': '44242', 'name': 'Data Structures', 'group': 2},
 {'id': '44249',
  'name': 'Building Blocks of Theoretical Computer Science',
  'group': 2},
 {'id': '44296', 'name': 'Professional Development Seminar', 'group': 2},
 {'id': '44300', 'name': 'Special Offering', 'gro

In [18]:
prereqs

{'44100': [],
 '44101': [],
 '44125': [],
 '44130': [],
 '44141': ['44101', '44130'],
 '44144': ['44101', '44130', '62130'],
 '44149': ['17117', '17120'],
 '44155': [],
 '44181': ['44101', '44130'],
 '44200': [],
 '44241': ['44141'],
 '44242': ['44241', '17114', '17117', '17118', '17120'],
 '44249': ['44141', '17117', '17118', '17120'],
 '44296': ['44141'],
 '44300': [],
 '44312': ['44101', '44130', '62130'],
 '44317': ['54313', '44101', '44130'],
 '44320': ['44101', '44130', '62130'],
 '44330': ['44101', '44130', '62130'],
 '44334': ['44144'],
 '44335': ['44334', '44141', '44144'],
 '44336': ['44101', '44130', '62130'],
 '44337': ['44336'],
 '44340': ['44101', '44130', '62130'],
 '44343': ['44101', '44130', '62130'],
 '44345': ['44242'],
 '44346': ['44101', '44130', '62130'],
 '44349': ['44242', '44249'],
 '44351': ['44242'],
 '44356': ['44130', '44101', '44542', '44542'],
 '44359': ['44356'],
 '44382': ['44242', '44356'],
 '44386': ['44241', '44296', '44356'],
 '44400': [],
 '44410':

In [23]:
curls

{'44100': 'http://nwmissouri.smartcatalogiq.com/en/2020-2021/Undergraduate-Catalog/Courses/CSIS-Computer-Science-Info-Systems/40000/CSIS-44100',
 '44101': 'http://nwmissouri.smartcatalogiq.com/en/2020-2021/Undergraduate-Catalog/Courses/CSIS-Computer-Science-Info-Systems/40000/CSIS-44101',
 '44125': 'http://nwmissouri.smartcatalogiq.com/en/2020-2021/Undergraduate-Catalog/Courses/CSIS-Computer-Science-Info-Systems/40000/CSIS-44125',
 '44130': 'http://nwmissouri.smartcatalogiq.com/en/2020-2021/Undergraduate-Catalog/Courses/CSIS-Computer-Science-Info-Systems/40000/CSIS-44130',
 '44141': 'http://nwmissouri.smartcatalogiq.com/en/2020-2021/Undergraduate-Catalog/Courses/CSIS-Computer-Science-Info-Systems/40000/CSIS-44141',
 '44144': 'http://nwmissouri.smartcatalogiq.com/en/2020-2021/Undergraduate-Catalog/Courses/CSIS-Computer-Science-Info-Systems/40000/CSIS-44144',
 '44149': 'http://nwmissouri.smartcatalogiq.com/en/2020-2021/Undergraduate-Catalog/Courses/CSIS-Computer-Science-Info-Systems/4000

In [24]:
p = requests.get('http://nwmissouri.smartcatalogiq.com/en/2020-2021/Undergraduate-Catalog/Courses')
s = BeautifulSoup(p.text, 'html5lib')

In [27]:
s.find_all('ul', {'class': 'sc-child-item-links'})

[<ul class="sc-child-item-links">
 	<li><a href="/en/2020-2021/Undergraduate-Catalog/Courses/ACCT-Accounting">ACCT - Accounting</a></li><li><a href="/en/2020-2021/Undergraduate-Catalog/Courses/AGRI-Agriculture">AGRI - Agriculture</a></li><li><a href="/en/2020-2021/Undergraduate-Catalog/Courses/ARSC-College-of-Arts-and-Sciences">ARSC - College of Arts and Sciences</a></li><li><a href="/en/2020-2021/Undergraduate-Catalog/Courses/ART-Art">ART - Art</a></li><li><a href="/en/2020-2021/Undergraduate-Catalog/Courses/BIOL-Biological-Sciences">BIOL - Biological Sciences</a></li><li><a href="/en/2020-2021/Undergraduate-Catalog/Courses/CHEM-Chemistry">CHEM - Chemistry</a></li><li><a href="/en/2020-2021/Undergraduate-Catalog/Courses/COM-Communication">COM - Communication</a></li><li><a href="/en/2020-2021/Undergraduate-Catalog/Courses/CORE-Northwest-Core-Interdisciplinary-Engagement">CORE - Northwest Core Interdisciplinary Engagement</a></li><li><a href="/en/2020-2021/Undergraduate-Catalog/Courses